#### Import and format the data

In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Read the data from a CSV file
data = pd.read_csv('P6\P6-C_230707101604_U1655.CSV', skiprows=range(0, 51), usecols=[0,1,2]) 
barom = pd.read_csv('Barometric_Boi_230707101832_P3959.CSV', skiprows=range(0, 51), usecols=[0,1,2])

# Dropping the last row
data = data.drop(data.index[-1])
barom = barom.drop(barom.index[-1])

# Convert the 'DateTime' column to datetime type if it's not already
data['Date/time'] = pd.to_datetime(data['Date/time'])
barom['Date/time'] = pd.to_datetime(barom['Date/time'])

# Set 'DateTime' column as the index
data.set_index('Date/time', inplace=True)
barom.set_index('Date/time', inplace=True)


#### Interpolation so that time stamps will match (using the resample method from pandas)

In [79]:
# Resample the DataFrame to 5-minute intervals and interpolate missing values
data_resampled = data.resample('5T').interpolate()
barom_resampled = barom.resample('5T').interpolate()

# Reset the index to obtain a DataFrame with 'DateTime' as a column
data_resampled.reset_index(inplace=True)
barom_resampled.reset_index(inplace=True)


#### Merge both barom and data df to match the same dates and time

In [80]:
# Merge the 'data' and 'barom' DataFrames on the common columns
merged_data = pd.merge(data_resampled, barom_resampled[['Date/time', 'Pressure[cmH2O]']], on='Date/time', how='left')


# Change the pressure column names
merged_data.rename(columns={'Pressure[cmH2O]_y': 'Barometric'}, inplace=True)
merged_data.rename(columns={'Pressure[cmH2O]_x': 'Pressure'}, inplace=True)

# Drop all rows that have a Nan
merged_data.dropna(inplace=True)

# Print the merged DataFrame
print(merged_data)

                 Date/time    Pressure  Temperature[°C]  Barometric
0      2022-07-12 16:00:00  750.817000        15.263000  755.425000
1      2022-07-12 16:05:00  750.758667        15.243000  755.308333
2      2022-07-12 16:10:00  750.700333        15.223000  755.191667
3      2022-07-12 16:15:00  750.642000        15.203000  755.075000
4      2022-07-12 16:20:00  750.583667        15.016333  755.133333
...                    ...         ...              ...         ...
101318 2023-06-29 11:10:00  748.833333        11.885667  752.761333
101319 2023-06-29 11:15:00  748.775000        11.927000  752.742000
101320 2023-06-29 11:20:00  748.833333        11.904667  752.644667
101321 2023-06-29 11:25:00  748.891667        11.882333  752.547333
101322 2023-06-29 11:30:00  748.950000        11.860000  752.450000

[101323 rows x 4 columns]


#### Compensate the pressure with the barometric and temperature

In [81]:
# Get the parameters
d_transducer = 0.4   # Depth of the pressure transducer in m - this value changes with each piezometer

# Convert the pressure from cmH2O to Pascals
barom_pa = merged_data['Barometric'] * 98.0665
pressure_pa = merged_data['Pressure'] * 98.0665

# Calculate the water density at the measured temperature
rho = 1000 / (1 + 0.00021 * (merged_data['Temperature[°C]'] - 4))

# Calculate the water depth
merged_data['Depth'] = (pressure_pa - barom_pa) / (rho * 9.81) + (0.5 - d_transducer)

# Save as csv in the compensated flow depth folder
output_folder = "../compensated-flow-depth"
merged_data.to_csv(os.path.join(output_folder, "P6(2).csv"), index=False)

In [82]:
merged_data

,Date/time,Pressure,Temperature[°C],Barometric,Depth
0,2022-07-12 16:00:00,750.817000,15.263000,755.425000,0.053827
1,2022-07-12 16:05:00,750.758667,15.243000,755.308333,0.054411
2,2022-07-12 16:10:00,750.700333,15.223000,755.191667,0.054996
3,2022-07-12 16:15:00,750.642000,15.203000,755.075000,0.055581
4,2022-07-12 16:20:00,750.583667,15.016333,755.133333,0.054414
...,...,...,...,...,...
101318,2023-06-29 11:10:00,748.833333,11.885667,752.761333,0.060668
101319,2023-06-29 11:15:00,748.775000,11.927000,752.742000,0.060278
101320,2023-06-29 11:20:00,748.833333,11.904667,752.644667,0.061836
101321,2023-06-29 11:25:00,748.891667,11.882333,752.547333,0.063395
